# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# obtain csv file and set as DataFrame
weather_file = "../output_data/weather_data_file.csv"
weather_df = pd.read_csv(weather_file)
weather_df



,City,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
0,a coruna,43.3713,-8.3960,63.93,90,0,3.00,ES,1630388641
1,abalak,58.1286,68.5944,51.37,62,88,7.92,RU,1630388714
2,abu dhabi,24.4667,54.3667,93.31,43,0,11.50,AE,1630388534
3,ada,43.4166,-116.1262,72.21,29,1,6.91,US,1630388752
4,adrar,20.5022,-10.0711,90.12,22,16,9.48,MR,1630388625
...,...,...,...,...,...,...,...,...,...
550,yumen,40.2833,97.2000,81.91,18,13,10.31,CN,1630388627
551,zhangye,38.9342,100.4517,75.94,33,81,2.84,CN,1630388537
552,zheleznodorozhnyy,55.7503,38.0042,63.12,68,93,8.25,RU,1630388761
553,zhigansk,66.7697,123.3711,46.15,57,95,7.96,RU,1630388664


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[["Lat","Lng"]].astype(float)
weight = weather_df["Humidity"].astype(float)

In [5]:
# Create Heatmaap
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '2px solid black',
    'padding': '6px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=(44.0, -5.0), zoom_level=1)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weight))
fig

Figure(layout=FigureLayout(border='2px solid black', height='400px', margin='0 auto 0 auto', padding='6px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down cities to fit 
# Ideal weather


weather_df["Max_temp"].max()

new_weather_df = pd.DataFrame(weather_df.loc[(weather_df["Max_temp"] > 60) & (weather_df["Max_temp"] < 70) & (weather_df["Cloudiness"] == 20), :])
new_weather_df.set_index("City")

,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
City,,,,,,,,
lisbon,38.7167,-9.1333,69.06,85,20,4.61,PT,1630388584
roma,43.2128,-75.4557,69.66,90,20,0.00,US,1630388584


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create new dataframe called hotel_df

hotel_df = pd.DataFrame(new_weather_df)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date,Hotel Name
268,lisbon,38.7167,-9.1333,69.06,85,20,4.61,PT,1630388584,
414,roma,43.2128,-75.4557,69.66,90,20,0.00,US,1630388584,


In [8]:
# set parameter to search for hotel witihn 5000 meters an dfind coordinates

def geocode(address):
    '''
    This is function that takes a string address and returns the latitude and longitude
    '''
    gparams = dict(address=address, key=g_key)
    # Build the endpoint URL
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json"
    geo_data = requests.get(target_url, params=gparams).json()
    try:
        lat = geo_data["results"][0]["geometry"]["location"]["lat"]
        lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    except:
        lat = None
        lng = None
    return lat, lng

In [9]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key

#print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [10]:
hotel_df

,City,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date,Hotel Name
268,lisbon,38.7167,-9.1333,69.06,85,20,4.61,PT,1630388584,Internacional Design Hotel
414,roma,43.2128,-75.4557,69.66,90,20,0.00,US,1630388584,Days Inn by Wyndham Utica


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(info_box_content=hotel_info, locations=locations)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='2px solid black', height='400px', margin='0 auto 0 auto', padding='6px', wi…